In [22]:
import pandas as pd
import numpy as np
FUP = pd.read_csv("../data/B3_AllFandU.csv").fillna(0)
FUP = FUP[FUP['Year']<=2021][FUT['Round']>=1]
p_year = 2014
if p_year == 2020:
    print("No Bracket in 2020")
if p_year != 2020:

    FUPN = FUP.select_dtypes(exclude=['object'])

    # Split the dataframe into x and y
    expl = FUPN.drop(['AFScore', 'AUScore'], axis=1)
    respF = FUPN[['Year','AFScore']]
    respU = FUPN[['Year','AUScore']]

    # Tran and Test
    X_train = expl[expl['Year'] != p_year]
    y_train_F = respF[respF['Year'] != p_year]['AFScore'].to_numpy() 
    y_train_U = respU[respU['Year'] != p_year]['AUScore'].to_numpy() 
    X_test = expl[expl['Year'] == p_year]
    y_test_F = respF[respF['Year'] == p_year]['AFScore'].to_numpy()
    y_test_U = respU[respU['Year'] == p_year]['AUScore'].to_numpy()

    # Make Model
    from sklearn.linear_model import ElasticNet
    enet = ElasticNet(alpha=.2,l1_ratio=.5)
    enet.fit(X_train,y_train_F)
    y_pred_F = enet.predict(X_test)
    enet.fit(X_train,y_train_U)
    y_pred_U = enet.predict(X_test)

    # How did the model do
    FUT = FUP[FUP['Year']==p_year]
    FUT = FUT.iloc[:,0:10]
    FUT['PFScore'] = y_pred_F
    FUT['PUScore'] = y_pred_U
    
    FUT['AWTeam'] = np.where(FUT['AFScore']>=FUT['AUScore'],FUT['AFTeam'],FUT['AUTeam'])
    FUT['PWTeam'] = np.where(FUT['PFScore']>=FUT['PUScore'],FUT['AFTeam'],FUT['AUTeam'])
    FUT.index = FUT.Game
    FUT['ESPN'] = np.where(FUT['AWTeam'] == FUT['PWTeam'],10*2**(FUT['Round']-1),0)
    #FUT['ESPN'] = (((FUT['AFScore'])>=(FUT['AUScore']))==((FUT['PFScore'])>=(FUT['PUScore']))).astype('uint8')*10*2**(FUT['Round']-1)
    #TESPN = FUT['ESPN'].sum()

    #st.markdown('Total ESPN Score: '+str(TESPN))
    #st.dataframe(FUT)

TypeError: '>=' not supported between instances of 'str' and 'int'